In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [17]:
var_QC_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/annotation_20201012/ukb24983_cal_hla_cnv.var_QC.tsv.gz'
cal_annot_f <- '/oak/stanford/groups/mrivas/ukbb24983/cal/annotation_20201002/ukb24983_cal_cALL_v2_hg19.vep101-loftee.Csq.tsv.gz'
ld_map_f <- '/oak/stanford/groups/mrivas/ukbb24983/array_imp_combined_no_cnv/ldmap/ldmap_20200928/ukb24983_cal_hla_imp.white_british.ld_map.tsv.gz'

QC_passed_lst_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/ld_indep_20201015/ukb24983_cal_hla_cnv.var_QCed.tsv'
QC_passed_ld_map_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/ld_indep_20201015/ukb24983_cal_hla_cnv.var_QCed.ld_map.tsv'


In [3]:
cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}

fread_CHROM <- function(f){
    fread(cmd=paste(cat_or_zcat(f), f), colClasses = c('#CHROM'='character')) %>% rename('CHROM'='#CHROM')
}


In [4]:
var_QC_f %>% fread_CHROM() %>% select(CHROM, POS, ID, REF, ALT, FILTER, geno_data_source, array) -> var_QC_df
cal_annot_f %>% fread_CHROM() %>% select(ID, Csq) -> cal_annot_df


In [11]:
var_QC_df %>% left_join(cal_annot_df, by='ID') %>% filter(FILTER == '.', geno_data_source == 'cal') -> df


In [12]:
df %>% count(Csq)

Csq,n
<chr>,<int>
intron,286561
others,263849
pav,74103
pcv,9655
ptv,23891
utr,11200


In [13]:
df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(QC_passed_lst_f, sep='\t', na = "NA", quote=F)
